# 4.3.1 CNN 텍스트 유사도 분석 모델

In [1]:
# 필요한 라이브러리 선언
import tensorflow.compat.v1 as tf
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

import json

# Parameter & Directory setup

In [2]:
# 필요한 전역변수 선언
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_Q1_DATA_FILE = 'train_q1.npy'
TRAIN_Q2_DATA_FILE = 'train_q2.npy'
TRAIN_LABEL_DATA_FILE = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

TEST_SPLIT = 0.1
RNG_SEED = 13371447

In [3]:
EPOCH=1
BATCH_SIZE=1024

MAX_SEQUENCE_LENGTH = 31

WORD_EMBEDDING_DIM = 100
CONV_FEATURE_DIM = 300
CONV_OUTPUT_DIM = 128
CONV_WINDOW_SIZE = 3
SIMILARITY_DENSE_FEATURE_DIM = 200

prepro_configs = None

with open(DATA_IN_PATH + DATA_CONFIGS, 'r') as f:
    prepro_configs = json.load(f)
    
VOCAB_SIZE = prepro_configs['vocab_size']

# Load Dataset

In [4]:
# 필요한 데이터 불러오기
q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

In [5]:
# q1,q2데이터 하나로 묶기
X = np.stack((q1_data, q2_data), axis=1)
# 라벨값 리스트로 생성
y = labels
# 학습 데이터, 평가 데이터 나누기
train_X, eval_X, train_y, eval_y = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RNG_SEED)

# 다시 두 질문 나누기
train_Q1 = train_X[:,0]
train_Q2 = train_X[:,1]
eval_Q1 = eval_X[:,0]
eval_Q2 = eval_X[:,1]

In [6]:
# 매핑 함수
# 인자: 질문, 대상 질문, 라벨 값
def rearrange(base, hypothesis, label):
    # 입력받은 두 인자를 하나의 딕셔너리 형태의 입력 값으로 바꾼다
    features = {"x1": base, "x2": hypothesis}
    # 딕셔너리와 라벨을 리턴하는 구조
    return features, label

# 학습을 위한 입력 함수
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((train_Q1, train_Q2, train_y))
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(rearrange)
    dataset = dataset.repeat(EPOCH)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()
# 평가를 위한 입력 함수
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((eval_Q1, eval_Q2, eval_y))
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(rearrange)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

# Model setup

In [7]:
# CNN블록함수(합성곱 신경망, 풀링, Dense를 하나로 합친 형태)
# 인자: 입력값, 이름
def basic_conv_sementic_network(inputs, name): 
    # Conv1D 사용
    conv_layer = tf.keras.layers.Conv1D(CONV_FEATURE_DIM, 
                                        CONV_WINDOW_SIZE, 
                                        activation=tf.nn.relu, 
                                        name=name + 'conv_1d',
                                        padding='same')(inputs)
    
    # MaxPooling1D 객체 활용
    max_pool_layer = tf.keras.layers.MaxPool1D(MAX_SEQUENCE_LENGTH, 
                                               1)(conv_layer)
    
    # 합성곱과 맥스풀링을 적용한 값에 대해 차원을 바꾸기 위해 Dense층을 통과시킨다.
    output_layer = tf.keras.layers.Dense(CONV_OUTPUT_DIM, 
                                         activation=tf.nn.relu,
                                         name=name + 'dense')(max_pool_layer)
    output_layer = tf.squeeze(output_layer, 1)
    
    # 결괏값 리턴
    return output_layer

In [8]:
# 모델 함수
# 인자: 입력값, 라벨, 모델함수가 사용된 모드
def model_fn(features, labels, mode):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    # 입력값을 임베딩 처리
    embedding = tf.keras.layers.Embedding(VOCAB_SIZE,
                                          WORD_EMBEDDING_DIM)
    # 기준 문장 임베딩 벡터로 전환
    base_embedded_matrix = embedding(features['x1'])
    # 대상 문장 임베딩 벡터로 전환
    hypothesis_embedded_matrix = embedding(features['x2'])
    
    # 기준 문장, 대상 문장 각각 드롭아웃 적용
    base_embedded_matrix = tf.keras.layers.Dropout(0.2)(base_embedded_matrix)
    hypothesis_embedded_matrix = tf.keras.layers.Dropout(0.2)(hypothesis_embedded_matrix)  
    
    # 기준 문장, 대상문장 각각 CNN 블록 함수에 적용
    base_sementic_matrix = basic_conv_sementic_network(base_embedded_matrix, 'base')
    hypothesis_sementic_matrix = basic_conv_sementic_network(hypothesis_embedded_matrix, 'hypothesis')  
    
    # 두 질문을 하나의 벡터로 만든다
    merged_matrix = tf.concat([base_sementic_matrix, hypothesis_sementic_matrix], -1)

    # 유사도 측정(Dense층 활용)
    similarity_dense_layer = tf.keras.layers.Dense(SIMILARITY_DENSE_FEATURE_DIM,
                                             activation=tf.nn.relu)(merged_matrix)
    
    # 드롭 아웃 적용
    similarity_dense_layer = tf.keras.layers.Dropout(0.2)(similarity_dense_layer)    
    
    # 마지막 Dense층은 1차원으로 설정 
    logit_layer = tf.keras.layers.Dense(1)(similarity_dense_layer)
    # 라벨과 비교해서 학습하기 위해 squeeze 함수 적용
    logit_layer = tf.squeeze(logit_layer, 1)
    # 예측값을 위한 시그모이드 함수 사용
    similarity = tf.nn.sigmoid(logit_layer)
    
    # 예측모드
    if PREDICT:
        # 예측 값 리턴
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  predictions={
                      'is_duplicate':similarity
                  })
    
    # 시그모이드 적용 전 logits 값과 라벨을 통해 손실값 계산
    loss = tf.losses.sigmoid_cross_entropy(labels, logit_layer)

    # 평가모드
    if EVAL:
        # 정확도 계산
        # 유사도 값을 round함수를 통해 반올림 한다.
        accuracy = tf.metrics.accuracy(labels, tf.round(similarity))
        # 정확도 리턴
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  eval_metric_ops= {'acc': accuracy},
                  loss=loss)
    
    # 학습모드
    if TRAIN:
        # 학습이 몇번 진행됬는지 확인
        global_step = tf.train.get_global_step()
        # 가중치 최적화를 위해 AdamOptimizer를 통한 손실값 적용
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)
        
        # 옵티마이저와 손실값 리턴
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  train_op=train_op,
                  loss=loss)

# Start training & Eval

In [9]:
os.environ["CUDA_VISIBLE_DEVICES"]="6" #For TEST

model_dir = os.path.join(os.getcwd(), DATA_OUT_PATH + "checkpoint/cnn/")
os.makedirs(model_dir, exist_ok=True)
# 에스티메이터 객체 생성
est = tf.estimator.Estimator(model_fn, model_dir=model_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/Users/sinseongjin/github/tensorflow-ml-nlp/5.TEXT_SIM/./data_out/checkpoint/cnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10f70fd30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
# 모델 학습
est.train(train_input_fn) #train

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /Users/sinseongjin/github/tensorflow-ml-nlp/5.TEXT_SIM/./data_out/checkpoint/cnn/model.ckpt.
INFO:tensorflow:loss = 0.69333047, step = 1
INFO:tensorflow:global_step/sec: 1.60758
INFO:tensorflow:loss = 0.54390585, step = 101 (62.208 sec)
INFO:tensorflow:global_step/sec: 1.52619
INFO:tensorflow:loss = 0.52071404, step = 201 (65.519 sec)
INFO:tensorflow:Saving checkpoints for 263 into /Users/sinseongjin/github/tensorflow-ml-nlp/5.TEXT_SIM/./data_out/checkpoint/cnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.47354826.


In [12]:
# 모델 검증
est.evaluate(eval_input_fn) #eval

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-20-02:25:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/sinseongjin/github/tensorflow-ml-nlp/5.TEXT_SIM/./data_out/checkpoint/cnn/model.ckpt-263
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-20-02:25:33
INFO:tensorflow:Saving dict for global step 263: acc = 0.75828224, global_step = 263, loss = 0.495392
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 263: /Users/sinseongjin/github/tensorflow-ml-nlp/5.TEXT_SIM/./data_out/checkpoint/cnn/model.ckpt-263


{'acc': 0.75828224, 'loss': 0.495392, 'global_step': 263}

# Load test dataset & create submit dataset to kaggle

In [15]:
# 캐글에 제출할 결과 만들기

# 전처리한 평가데이터 불러오기
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'
TEST_ID_DATA_FILE = 'test_id.npy'

# 넘파이 배열로 열기
test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'))

In [16]:
# 입력 함수 생성
# 에스티메이터의 numpy_input_fn함수를 사용하여 두 질문 데이터를 하나의 딕셔너리로 병합
# shuffle이 False인 이유는 순서대로 들어가야 하기 때문
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x1":test_q1_data,
                                                         "x2":test_q2_data},
                                                      shuffle=False)
# 에스티메이터의 predict함수를 사용하여 예측 결과 생성
predictions = np.array([p['is_duplicate'] for p in est.predict(input_fn=predict_input_fn)])

# 결과값 판다스 데이터프레임으로 생성
output = pd.DataFrame( data={"test_id":test_id_data, "is_duplicate": list(predictions)} )
# 결과값 저장
output.to_csv("cnn_predict.csv", index=False, quoting=3)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/sinseongjin/github/tensorflow-ml-nlp/5.TEXT_SIM/./data_out/checkpoint/cnn/model.ckpt-263
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2345796
